In [3]:
import os
from PIL import Image
import pandas as pd
from pandas import read_csv
import numpy as np

img_folder = "../Yoio_Park_Proof"
target_size = (512, 512)
label_data = read_csv("../model_julius/balanced_labels.csv", sep=";")

# Function to load and resize an image
def load_image(img_path, target_size):
    with Image.open(img_path) as img:
        img = img.resize(target_size)
        img = img.convert('RGB')  # Ensure image is in RGB format
        return np.array(img)

# Read images and their corresponding labels
image_data = []
image_labels = []

for idx, row in label_data.iterrows():
    img_name = row['Image_name']
    img_path = os.path.join(img_folder, img_name)
    
    if os.path.exists(img_path):
        img_array = load_image(img_path, target_size)
        image_data.append(img_array)
        image_labels.append(row[1])  # Assuming labels are in columns after 'Image'
        print(f"Loaded image {img_name} with label {row[1]}", end='\r')
    else:
        print(f"Warning: Image {img_name} not found in folder {img_folder}")



C:\Users\juliu\AppData\Local\Temp\ipykernel_24072\32932425.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_name = row[0]
C:\Users\juliu\AppData\Local\Temp\ipykernel_24072\32932425.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_labels.append(row[1])  # Assuming labels are in columns after 'Image'
C:\Users\juliu\AppData\Local\Temp\ipykernel_24072\32932425.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(

KeyboardInterrupt: 

In [1]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.1,
    fill_mode='nearest'
)

KeyboardInterrupt: 

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(image_data, image_labels, test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [4]:
import pickle

with open('../model_julius/test_img.pickle', 'wb') as f:
    pickle.dump((X_test, y_test), f)

In [5]:
#generate augmented images
batches = 2000
augmented_images = []
augmented_labels = []

for i in range(batches):
    batch = next(datagen.flow(X_train, y_train, batch_size=1))
    augmented_images.append(batch[0][0])
    augmented_labels.append(batch[1][0])
    print(f"Generated {i+1} images", end='\r')

In [6]:
from tensorflow.keras.utils import to_categorical
#normalize the images
augmented_images = np.array(augmented_images)/255.0
augmented_labels = to_categorical(augmented_labels)

In [7]:
import pickle

with open('../model_julius/data_useable.pickle', 'wb') as f:
    pickle.dump((augmented_images, augmented_labels), f)